In [ ]:
import hashlib
import json
from time import time

# Define the Blockchain Class
class Blockchain:
    def __init__(self):
        self.chain = []
        self.current_transactions = []
        # Create the genesis block
        self.new_block(previous_hash='1', proof=100)
    
    def new_block(self, proof, previous_hash=None):
        block = {
            'index': len(self.chain) + 1,
            'timestamp': time(),
            'transactions': self.current_transactions,
            'proof': proof,
            'previous_hash': previous_hash or self.hash(self.chain[-1]),
        }
        # Reset the current list of transactions
        self.current_transactions = []
        self.chain.append(block)
        return block
    
    def new_transaction(self, sender, recipient, data):
        self.current_transactions.append({
            'sender': sender,
            'recipient': recipient,
            'data': data
        })
        return self.last_block['index'] + 1
    
    def hash(self, block):
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()
    
    @property
    def last_block(self):
        return self.chain[-1]
    
    def proof_of_work(self, last_proof):
        proof = 0
        while self.valid_proof(last_proof, proof) is False:
            proof += 1
        return proof
    
    def valid_proof(self, last_proof, proof):
        guess = f'{last_proof}{proof}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == "0000"  # Proof must start with 4 leading zeros

# Identity Verification System
class IdentityVerificationSystem:
    def __init__(self):
        self.blockchain = Blockchain()

    def create_identity(self, user_name, user_email, user_phone):
        """
        Create a new user identity and store it in the blockchain.
        User details will be hashed and added as a transaction.
        """
        user_details = {
            'name': user_name,
            'email': user_email,
            'phone': user_phone
        }
        
        # Hash the user details to maintain privacy
        hashed_user_details = hashlib.sha256(json.dumps(user_details, sort_keys=True).encode()).hexdigest()

        # Add the user identity transaction to the blockchain
        self.blockchain.new_transaction(
            sender="System",
            recipient=user_email,
            data=hashed_user_details
        )
        
        # Mine a new block to add this transaction to the blockchain
        last_proof = self.blockchain.last_block['proof']
        proof = self.blockchain.proof_of_work(last_proof)
        self.blockchain.new_block(proof, self.blockchain.hash(self.blockchain.last_block))
        print(f"Identity for {user_name} has been successfully created.")

    def verify_identity(self, user_name, user_email, user_phone):
        """
        Verify the identity of a user by checking if their hashed details exist on the blockchain.
        """
        user_details = {
            'name': user_name,
            'email': user_email,
            'phone': user_phone
        }
        
        # Hash the user details again for verification
        hashed_user_details = hashlib.sha256(json.dumps(user_details, sort_keys=True).encode()).hexdigest()

        # Search for the hashed details in the blockchain
        for block in self.blockchain.chain:
            for transaction in block['transactions']:
                if transaction['recipient'] == user_email and transaction['data'] == hashed_user_details:
                    print(f"Identity verified for {user_name}.")
                    return True
        print("Identity verification failed. Details not found.")
        return False

# Main function to test the blockchain-based identity verification
def main():
    system = IdentityVerificationSystem()
    
    # Create and verify identity
    system.create_identity("John Doe", "john.doe@example.com", "555-1234")
    system.verify_identity("John Doe", "john.doe@example.com", "555-1234")
    
    # Test invalid verification
    system.verify_identity("Jane Doe", "jane.doe@example.com", "555-5678")

if __name__ == "__main__":
    main()